# How to build a simple Chatbot with stored memory using LangChain
* Simple Chatbot LLM App.
    * Will be able to have a conversation.
    * Will remember previous interactions: will have memory.
    * Will be able to store memory in a json file.

A chatbot app is a software application designed to simulate conversation with human users. It uses artificial intelligence (AI) to understand what someone is saying and to respond with appropriate answers. These apps can be used for various purposes like customer support, providing information, or entertainment. Essentially, it's like texting with a program that can answer questions and help with tasks.

## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages.
    *  LLM Model is based around raw text.
* Chat History: allows Chat Model to remember previous interactions.



In [1]:
#Avoid legacy Langchain depreciation warning
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

In [2]:
#load my env variable
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
groq_api_key = os.environ["GROQ_API_KEY"]

In [5]:
#load our chat completion model
from langchain_groq import ChatGroq

chatbot = ChatGroq(model="mixtral-8x7b-32768")

In [6]:
#Let's check with simple prompt - human prompt
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favourite place is Switzerland"),
]

In [7]:
#Let's pass to the LLM with the help of invoke function
#Response in the form of AIMessage
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="Switzerland is a beautiful country located in the heart of Europe, known for its stunning landscapes, charming towns and villages, and rich culture. Here are a few things that make Switzerland a wonderful place to visit:\n\n1. Natural Beauty: Switzerland is a country of breathtaking natural beauty, with towering mountains, crystal-clear lakes, and lush green valleys. The Swiss Alps are a popular destination for hiking, skiing, and mountaineering, and offer some of the most stunning views in the world.\n2. Cultural Diversity: Switzerland is a federal state made up of 26 cantons, each with its own unique culture, language, and traditions. The country has four official languages: German, French, Italian, and Romansh, and visitors can experience a rich tapestry of cultural diversity as they travel from one region to another.\n3. Delicious Food: Swiss cuisine is renowned for its quality and variety. From hearty dishes like fondue and raclette to delicate pastries and choc

In [8]:
#Let's check our chatbot remembers our favourite place
chatbot.invoke([
    HumanMessage(content="Do you know my favourite place?")
])

AIMessage(content="I'm unable to determine your favorite place as I don't have access to your personal memories, experiences, or preferences. You can tell me about your favorite place if you'd like to share!", response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 15, 'total_tokens': 58, 'completion_time': 0.066939793, 'prompt_time': 0.002255984, 'queue_time': 0.030961621999999998, 'total_time': 0.069195777}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-caff8c54-72c0-49e5-bb16-c7eac4fc6c6d-0', usage_metadata={'input_tokens': 15, 'output_tokens': 43, 'total_tokens': 58})

In [9]:
#Let's add memory to our Chatbot

from langchain import LLMChain #legacy langchain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory #memory
from langchain.memory import FileChatMessageHistory

In [10]:
#Let's create memory object with the help of conversation buffer memory
#save the memory in a file - permanent memory
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True
)

In [11]:
#prompt template
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

In [12]:
#Create a chain
chain = LLMChain(
    llm=chatbot,
    prompt=prompt,
    memory=memory
)


In [13]:
#invoke
chain.invoke("hello")

{'content': 'hello',
 'messages': [],
 'text': "Hello! It's nice to meet you. Is there something you would like to talk about or ask me? I'm here to help with any questions you might have about writing, grammar, or language in general. Just let me know how I can assist you."}

In [14]:
chain.invoke("My name is Amar")

{'content': 'My name is Amar',
 'messages': [HumanMessage(content='hello'),
  AIMessage(content="Hello! It's nice to meet you. Is there something you would like to talk about or ask me? I'm here to help with any questions you might have about writing, grammar, or language in general. Just let me know how I can assist you.")],
 'text': "Hi Amar, it's nice to meet you! How can I help you today? Is there something you would like to ask or talk about related to writing, grammar, or language? I'm here to assist you in any way I can. Just let me know what's on your mind."}

In [15]:
chain.invoke('Do you know my name?')

{'content': 'Do you know my name?',
 'messages': [HumanMessage(content='hello'),
  AIMessage(content="Hello! It's nice to meet you. Is there something you would like to talk about or ask me? I'm here to help with any questions you might have about writing, grammar, or language in general. Just let me know how I can assist you."),
  HumanMessage(content='My name is Amar'),
  AIMessage(content="Hi Amar, it's nice to meet you! How can I help you today? Is there something you would like to ask or talk about related to writing, grammar, or language? I'm here to assist you in any way I can. Just let me know what's on your mind.")],
 'text': "Yes, you mentioned that your name is Amar earlier in our conversation. I'm here to assist you with any questions you might have about writing, grammar, or language. Is there something specific you would like to ask or discuss? I'm here to help."}